In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime, timedelta

import json
import openai



# Set the path of the chromedriver.exe
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open WhatsApp
driver.get('https://web.whatsapp.com/')

# Wait for the user to scan the QR code
wait = WebDriverWait(driver, 600)
print('Scan QR Code, And then Enter')
input()
print("Logged In")



Scan QR Code, And then Enter
Logged In


In [4]:
# Get the current date

current_date = datetime.now().date()
target = '"Bhaiya 😜😜🤗💛❤️ Pandey"' 
x_arg = '//span[contains(@title,' + target + ')]'

with open('secrets.json') as f:
    data = json.load(f)

openai.api_key = data['api_key']

messages = [
    {"role": "system", "content": "Personal assistsnt for queries, I am a software developer, python developer, data science and machine learning developer, I also take classes on these topics"},
]

del data

def clean_text(msg):
    return msg.replace('\n', '').replace('\ue008\ue007\ue008', '').replace('`', '').replace('(Keys.SHIFT)+(Keys.ENTER)+(Keys.SHIFT)', '')

def chat_response(input_message):
    global messages
    if input_message:
        messages.append({
            'role': 'user', 'content': input_message
        })
        chat = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo", messages = messages
        )

    reply = chat.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    return reply

last_message_text = ''
my_last_message = ''

if __name__ == "__main__":
    while True:
        try:
            # Wait for the specific contact to show up
            group_title = wait.until(EC.presence_of_element_located((By.XPATH, x_arg)))
            group_title.click()

            # Detect new message by checking the last message in the conversation
            all_messages = driver.find_elements(By.CSS_SELECTOR, "div._1BOF7._2AOIt")
            last_message = all_messages[-1]
            
            # Get the timestamp from the message
            timestamp_element = last_message.find_element(By.CSS_SELECTOR, "span.l7jjieqr.fewfhwl7")
            timestamp_str = timestamp_element.text  # "12:42 am"

            # Get the message text
            message_text_element = last_message.find_element(By.CSS_SELECTOR, "span._11JPr.selectable-text.copyable-text")
            message_text = message_text_element.text

            # Convert the timestamp to a datetime object
            timestamp = datetime.strptime(timestamp_str, "%I:%M %p")

            # Update the year, month and day of the timestamp to match the current date
            timestamp = timestamp.replace(year=current_date.year, month=current_date.month, day=current_date.day)

            # Get the current time
            current_time = datetime.now()

            
            # print((current_time - timestamp), repr(last_message_text), repr(my_last_message), repr(message_text))
            # If the message was sent less than 60 seconds ago, respond to it
            if (current_time - timestamp < timedelta(seconds=70)) and (clean_text(last_message_text)[:200] != clean_text(message_text)[:200]) and (clean_text(my_last_message)[:200] != clean_text(message_text)[:200]):
                print("messaging")
                my_message = chat_response(message_text).replace('\n', (Keys.SHIFT)+(Keys.ENTER)+(Keys.SHIFT))
                # my_message = 'Hi this is the message I want to send \n but this shouldnt go in the next line'.replace('\n', (Keys.SHIFT)+(Keys.ENTER)+(Keys.SHIFT))
                msg_box = driver.find_elements(By.CLASS_NAME, '_3Uu1_')[0]
                msg_box.send_keys(my_message + Keys.ENTER)
                last_message_text = clean_text(message_text)
                my_last_message = clean_text(my_message)
        except Exception as e:
            print("An error occurred: ", e)
            continue
        time.sleep(10)  # Wait for 10 seconds before checking for new messages

0:07:14.605416 '' '' "Yes, that is correct! This program will find the shortest value in a list of integers without using the built-in `min()` function in Python. \n\nDo you have any other questions or tasks related to Python or programming in general? I'm happy to help."
0:07:24.686560 '' '' "Yes, that is correct! This program will find the shortest value in a list of integers without using the built-in `min()` function in Python. \n\nDo you have any other questions or tasks related to Python or programming in general? I'm happy to help."
0:00:34.764808 '' '' 'A program to find the shortest value in a list of integers in python. Don’t use the min function'
messaging
0:00:56.516767 'A program to find the shortest value in a list of integers in python. Don’t use the min function' 'Sure, here\'s a Python program to find the shortest value in a list of integers without using the built-in min() function:pythondef find_shortest(lst):    """    Returns the shortest value in a list of integer

KeyboardInterrupt: 

In [46]:
chat_response('what is a GPU?')

'A GPU (Graphics Processing Unit) is a specialized processor designed to handle complex calculations needed for rendering graphics and images. Unlike a CPU (Central Processing Unit) which is designed to handle a broad range of tasks, a GPU is optimized for parallel processing, which makes it more effective at performing complex mathematical computations. GPUs are commonly used for applications such as video games, image and video processing, and machine learning, where large amounts of data need to be processed in parallel.'

In [ ]:
0:00:49.401032 'In statistics, a t-test is a type of hypothesis test that is used to determine if there is a significant difference between the means of two groups or sets of data. \n\nThe t-test is designed to compare the means of two groups by examining the difference between the means relative to the variability of both groups. Specifically, the t-test calculates a t-value, which is the difference between the means of the two groups divided by the standard error of the difference between the means. \n\nThe t-value is then compared to a critical value obtained from a t-distribution table to determine if the difference between the means is statistically significant at a given level of significance (usually 0.05 or 0.01). If the t-value is greater than the critical value, then …'

'...the null hypothesis (which states that there is no significant difference between the means of the two groups) is rejected. This means that there is evidence to suggest that there is a significant difference between the means of the two groups. On the other hand, if the t-value is less than the critical value, then the null hypothesis is not rejected and it is concluded that there is no significant difference between the means of the two groups. The t-test can be performed as either an independent samples t-test or a paired samples t-test, depending on the nature of the data being compared.' 

'...the null hypothesis (which states that there is no significant difference between the means of the two groups) is rejected. This means that there is evidence to suggest that there is a significant difference between the means of the two groups. On the other hand, if the t-value is less than the critical value, then the null hypothesis is not rejected and it is concluded that there is no significant difference between the means of the two groups. The t-test can be performed as either an independent samples t-test or a paired samples t-test, depending on the nature of the data being compared.'


In [ ]:
0:00:32.016704 'What is t test in statistics' 
'In statistics, a t-test is a type of hypothesis test that is used to determine if there is a significant difference between the means of two groups or sets of data. \n\nThe t-test is designed to compare the means of two groups by examining the difference between the means relative to the variability of both groups. Specifically, the t-test calculates a t-value, which is the difference between the means of the two groups divided by the standard error of the difference between the means. \n\nThe t-value is then compared to a critical value obtained from a t-distribution table to determine if the difference between the means is statistically significant at a given level of significance (usually 0.05 or 0.01). If the t-value is greater than the critical value, then the null hypothesis (which states that there is no significant difference between the means of the two groups) is rejected and it is concluded that the means are significantly different. If the t-value is less than the critical value, then the null hypothesis cannot be rejected and it is concluded that the means are not significantly different.' 

'In statistics, a t-test is a type of hypothesis test that is used to determine if there is a significant difference between the means of two groups or sets of data. \n\nThe t-test is designed to compare the means of two groups by examining the difference between the means relative to the variability of both groups. Specifically, the t-test calculates a t-value, which is the difference between the means of the two groups divided by the standard error of the difference between the means. \n\nThe t-value is then compared to a critical value obtained from a t-distribution table to determine if the difference between the means is statistically significant at a given level of significance (usually 0.05 or 0.01). If the t-value is greater than the critical value, then …'


In [ ]:
len('''Sure, I can help you with that.

Here is a Python program that finds the shortest value in a list of integers without using the built-in `min` function:


my_list = [5, 3, 8, 2, 9]

shortest = my_list[0]

for i in range(1, len(my_list)):
    if my_list[i] < shortest:
        shortest = my_list[i]

print("The shortest value in the list is:", shortest)


Here is what the above code does:

1. We start by defining our list of integers. In this example, we have used the list `[5, 3, 8, 2, 9]`.''')

489